In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 11.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from deap import base, creator, tools
import random
import copy

# === Load Dataset ===
df = pd.read_csv("/dataset/balanced100k_2018.csv")

# Drop unnecessary columns

<ipython-input-3-cc48c3ab137f>:10: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/network/balanced100k_2018.csv")


In [4]:
df.drop(labels=[ 'Flow ID', 'Src IP', 'Src Port',
              'Dst IP', 'Dst Port', 'Protocol', 'Timestamp'], axis=1,inplace=True)
df = df[df['Flow Duration'] != 'Flow Duration']

In [5]:
ALL_FEATURES = df.columns.tolist()
ALL_FEATURES.remove("Label")


In [6]:
df[ALL_FEATURES] = df[ALL_FEATURES].apply(pd.to_numeric, errors='coerce')
df[ALL_FEATURES] = df[ALL_FEATURES].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=ALL_FEATURES)

In [ ]:
df

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1172999,3,4,20.0,964.0,20.0,0.0,6.666667,11.547005,964.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,DDoS attacks-LOIC-HTTP
1,2378,3,1,31.0,0.0,31.0,0.0,10.333333,17.897858,0.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Benign
2,9021,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,32,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,DoS attacks-Hulk
3,59834166,9,15,548.0,9245.0,183.0,0.0,60.888889,91.333790,1460.0,...,20,100208.666667,16720.383498,118772.0,86331.0,1.984451e+07,9.559293e+06,25480312.0,8807222.0,Benign
4,10819,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Bot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599992,10952,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Bot
599993,499,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Bot
599994,77952,2,1,74.0,126.0,74.0,0.0,37.000000,52.325902,126.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Benign
599995,13578,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,Bot


In [ ]:
nan_rows = df[feature_columns].isna().sum(axis=1)
print("Number of rows with missing values:", (nan_rows > 0).sum())

# Optionally, show these rows
print(df[nan_rows > 0])

In [7]:



# Features and Label
ALL_FEATURES = df.columns.tolist()
ALL_FEATURES.remove("Label")
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df[ALL_FEATURES])
y = df["Label"].values

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense, Activation

In [11]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

# === Reshape for Conv1D ===
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
from sklearn.model_selection import train_test_split

# === Split into Train/Test ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)


In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)
from sklearn.model_selection import train_test_split

# === Reshape for Conv1D ===
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# === Split into Train/Test ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)

# === Define CNN Model ===
# model = Sequential()
# model.add(Conv1D(64, kernel_size=3, padding='same', input_shape=(X_train.shape[1], 1)))
# model.add(Activation('relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))
model = tf.keras.models.load_model("/model/100k_cnn_yeni_model.keras")



In [ ]:
from sklearn.model_selection import train_test_split

# Encode Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)

model.fit(X_train, y_train)
# === Train RandomForest Model ===
# model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
# model.fit(X_scaled, y_encoded)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [15]:
attack_labels = [
    "Bot", "DDoS attacks-LOIC-HTTP", "DoS attacks-GoldenEye", "DoS attacks-Hulk",
    "DoS attacks-SlowHTTPTest", "DoS attacks-Slowloris", "FTP-BruteForce", "Infilteration", "SSH-Bruteforce"
]
attack_class_indices = [np.where(label_encoder.classes_ == attack)[0][0] for attack in attack_labels]
def model_predict_prob_batch(samples):
    samples = np.array(samples).reshape(len(samples), -1, 1)
    return model.predict(samples, verbose=0)

def attack_probability(pred_vector):
    return sum(pred_vector[i] for i in attack_class_indices)

# === Fitness Function (BATCH compatible) ===
def evaluate_population(population):
    samples_batch = []
    for individual in population:
        full_sample = original_sample.copy()
        for i, idx in enumerate(MUT_IDX):
            full_sample[idx] = individual[i]
        samples_batch.append(full_sample)

    pred_vectors = model_predict_prob_batch(samples_batch)

    fitnesses = []
    for individual, pred_vector in zip(population, pred_vectors):
        attack_prob = attack_probability(pred_vector)
        distortion = np.linalg.norm(np.array(individual) - original_sample[MUT_IDX])
        fitnesses.append((1.0 - attack_prob, -distortion))

    return fitnesses

# === GA Preparation ===
creator.create("FitnessMulti", base.Fitness, weights=(0.8, 1.2))
creator.create("Individual", list, fitness=creator.FitnessMulti)

# === run_ga (updated for batch fitness) ===
def run_ga(original_sample, generations=10, population_size=50):
    toolbox = base.Toolbox()
    for i in range(len(MUT_IDX)):
        low, high = 0.0, 1.0
        center = original_sample[MUT_IDX[i]]
        toolbox.register(f"attr_float_{i}", bounded_uniform, low, high, center, 0.2)

    toolbox.register("individual", tools.initCycle, creator.Individual,
                     tuple(toolbox.__getattribute__(f"attr_float_{i}") for i in range(len(MUT_IDX))), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.05, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=population_size)

    for gen in range(generations):
        fitnesses = evaluate_population(population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.6:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.3:
                toolbox.mutate(mutant)
                del mutant.fitness.values
                for i in range(len(mutant)):
                    mutant[i] = np.clip(mutant[i], 0.0, 1.0)

        invalid = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = evaluate_population(invalid)
        for ind, fit in zip(invalid, fitnesses):
            ind.fitness.values = fit

        elite = tools.selBest(population, k=1)
        population[:] = offspring[:-1] + elite

    best = tools.selBest(population, k=1)[0]
    print(f"Gen {gen+1}: Best fitness = {best.fitness.values}")

    return best
# === Run GA and Collect Statistics ===


In [15]:
model.save("/content/drive/MyDrive/network/100k_cnn_yeni_model.keras")

In [19]:

MUTATED_FEATURES = ['Fwd Pkt Len Mean',
    'Bwd Pkt Len Mean',
    'Pkt Len Mean',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Init Fwd Win Byts',
    'Init Bwd Win Byts']
MUT_IDX = [ALL_FEATURES.index(f) for f in MUTATED_FEATURES]
attack_indices = np.where(pd.Series(y).isin(attack_labels))[0]

statistics = []

for idx, row_idx in enumerate(attack_indices[300:500]):
    print(f"\n🚀 Mutating GA sample {idx+1}/5")

    original_sample = X_test[row_idx].copy()
    original_label = y_test[row_idx]
    pred_vector_before = model_predict_prob_vector(original_sample)
    attack_prob_before = attack_probability(pred_vector_before)
    predicted_label_before = np.argmax(pred_vector_before)

    best_ind = run_ga(original_sample)

    mutated_sample = original_sample.copy()
    for i, idx_feat in enumerate(MUT_IDX):
        mutated_sample[idx_feat] = best_ind[i]

    pred_vector_after = model_predict_prob_vector(mutated_sample)
    attack_prob_after = attack_probability(pred_vector_after)
    predicted_label_after = np.argmax(pred_vector_after)

    mutation_count = np.sum(original_sample.flatten() != mutated_sample.flatten())
    feature_change_ratio = mutation_count / len(ALL_FEATURES)
    distortion = np.linalg.norm(mutated_sample.flatten() - original_sample.flatten())

    statistics.append({
        "sample_id": idx,
        "original_label": original_label,
        "predicted_label_before": predicted_label_before,
        "predicted_label_after": predicted_label_after,
        "attack_prob_before": attack_prob_before,
        "attack_prob_after": attack_prob_after,
        "distortion": distortion,
        "mutation_count": mutation_count,
        "feature_change_ratio": feature_change_ratio,
        "original_features": original_sample.flatten().tolist(),
        "mutated_features": mutated_sample.flatten().tolist(),
        "attack_class_prob_vector_before": pred_vector_before.tolist(),
        "attack_class_prob_vector_after": pred_vector_after.tolist()
    })

# Save statistics
statistics_df = pd.DataFrame(statistics)
# statistics_df.to_csv("mutation_statistics_svm100_300.csv", index=False, sep=";")

print("✅ All adversarial samples and mutation statistics saved with RandomForest model!")



🚀 Mutating GA sample 1/5
Gen 10: Best fitness = (np.float32(0.8368591), np.float64(-0.25396743792612636))

🚀 Mutating GA sample 2/5
Gen 10: Best fitness = (np.float32(0.9968662), np.float64(-0.5171373761720052))

🚀 Mutating GA sample 3/5
Gen 10: Best fitness = (np.float32(0.6171936), np.float64(-0.3207526275065808))

🚀 Mutating GA sample 4/5
Gen 10: Best fitness = (np.float32(0.6446537), np.float64(-0.3849444697566695))

🚀 Mutating GA sample 5/5
Gen 10: Best fitness = (np.float32(0.42359352), np.float64(-0.4485109542011606))

🚀 Mutating GA sample 6/5
Gen 10: Best fitness = (np.float32(0.48888087), np.float64(-0.6350255745619926))

🚀 Mutating GA sample 7/5
Gen 10: Best fitness = (np.float32(0.9732973), np.float64(-0.5452240652017185))

🚀 Mutating GA sample 8/5
Gen 10: Best fitness = (np.float32(0.5915228), np.float64(-0.2496831357753601))

🚀 Mutating GA sample 9/5
Gen 10: Best fitness = (np.float32(0.63137555), np.float64(-0.3107462054544839))

🚀 Mutating GA sample 10/5
Gen 10: Best fi

In [ ]:
statistics_df.to_csv("/results/mutation_statistics_100k_cnn_yeni_200_2.csv", index=False, sep=";")
